In [1]:
import numpy as np
import pandas as pd
%matplotlib inline


In [2]:
nba_data = pd.DataFrame(pd.read_csv('./dataset/matches.csv'))
nba_data.columns = ["number", "Date", "Visitor Team", "VisitorPts", "Home Team", "HomePts","Score Type", "OT?", "Notes"]

nba_data = nba_data.drop(1230)#删除index为1230的这行,这行数据为空数据
nba_data.index = [i for i in range(len(nba_data))]#index重新赋值

standing = pd.read_csv('./dataset/standing.csv')#上一个赛季的成绩
standing = standing.rename(columns={'0':'Team','Unnamed: 0':'Rk','1':'Overall','2':'Home','3':'Road','4':'E',\
                                    '5':'W','6':'A','7':'C','8':'SE'})


In [3]:
from collections import defaultdict
last_win_dict = defaultdict(int)
nba_data['win_type'] = nba_data['HomePts'] > nba_data['VisitorPts']#增加一列,home队获胜则为1,否则为0
nba_data['HT last win'] = False#home上一次比赛是否胜利,胜利为true
nba_data['VT last win'] = False#visitor上一次比赛是否胜利,胜利为true
nba_data['last_home_Rk'] = 0#上一个赛季home的排名
nba_data['last_visitor_Rk'] = 0#上一个赛季visitor的排名
nba_data['last_home_or_visitor_win'] = False#上一次比赛home和visitor谁赢得比赛
nba_data['last_home_and_visitor_score_diff'] = 0
nba_data['home_to_last_distance'] = 0#home距离上一次比赛的时间间隔,0表示之前还没有比赛过,
nba_data['visitor_to_last_distance'] = 0#visitor距离上一次比赛的时间间隔,0表示之前还没有比赛过,
nba_data['date_diff_between_two_team'] = 0#该特征表示home team距离上一次比赛的时间间隔与visitor team距离上一次比赛的时间间隔的差值
temp_list = []
home_last_compete = defaultdict(int)
visitor_last_compete = defaultdict(int)

for index ,row in nba_data.iterrows():
    home_team = row['Home Team']
    visitor_team = row['Visitor Team']
    if home_team == 'New Orleans Pelicans':
        home_team = 'New Orleans Hornets'
    elif visitor_team == 'New Orleans Pelicans':
        visitor_team = 'New Orleans Hornets'
    row['HT last win'] = last_win_dict[home_team]#last_win_dict是一个字典,存放着每个队伍上一次比赛是否胜利,胜利为True
    row['VT last win'] = last_win_dict[visitor_team]
    last_win_dict[home_team] = row['win_type']
    last_win_dict[visitor_team] = not row['win_type']
    
    last_home_Rk = standing[standing['Team'] == home_team]['Rk'].values[0]
    last_visitor_Rk = standing[standing['Team'] == visitor_team]['Rk'].values[0]
    row['last_home_Rk'] = last_home_Rk
    row['last_visitor_Rk'] = last_visitor_Rk
    row['last_home_or_visitor_win'] = bool(last_home_Rk > last_visitor_Rk)#用bool表示
    row['last_home_and_visitor_score_diff'] = last_home_Rk - last_visitor_Rk#用离散值来代替
    
    if home_last_compete[home_team] != 0:
        row['home_to_last_distance'] = (row['number'] - home_last_compete[home_team])
    if visitor_last_compete[visitor_team] != 0:
        row['visitor_to_last_distance'] = (row['number'] - visitor_last_compete[visitor_team])
    home_last_compete[home_team] = row['number']
    visitor_last_compete[visitor_team] = row['number']
    
    temp_list.append(row['home_to_last_distance']-row['visitor_to_last_distance'])
    row['date_diff_between_two_team'] = (row['home_to_last_distance']-row['visitor_to_last_distance'])
    
    nba_data.ix[index] = row#将更新后的行赋给原来的dataframe

/home/lg-hu/anaconda3/envs/p35/lib/python3.5/site-packages/ipykernel_launcher.py:46: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [4]:
home_last_matches_win = defaultdict(int)#记录home team前五次比赛中胜利的次数
visitor_last_matches_win = defaultdict(int)#记录visitor team前五次比赛中胜利的次数
home_cnt = defaultdict(int)
visitor_cnt = defaultdict(int)
nba_data['home_last_matches_win']=0
nba_data['visitor_last_matches_win']=0

for index ,row in nba_data.iterrows():
    home_team = row['Home Team']
    visitor_team = row['Visitor Team']
    home_cnt[home_team] += 1
    visitor_cnt[visitor_team] += 1
    if home_cnt[home_team] == 5: 
        home_cnt[home_team] = 1 
        home_last_matches_win[home_team] = 0
    if visitor_cnt[visitor_team] == 5: 
        visitor_cnt[visitor_team]  = 1 
        visitor_last_matches_win[visitor_team] = 0
    
    if row['HT last win']:
        home_last_matches_win[home_team] += 1
    if row['VT last win']:
        visitor_last_matches_win[visitor_team] += 1
    row['home_last_matches_win'] = home_last_matches_win[home_team]
    row['visitor_last_matches_win'] = visitor_last_matches_win[visitor_team]
#     print cnt
    nba_data.ix[index] = row#将更新后的行赋给原来的dataframe


In [5]:
nba_data

,number,Date,Visitor Team,VisitorPts,Home Team,HomePts,Score Type,OT?,Notes,win_type,...,VT last win,last_home_Rk,last_visitor_Rk,last_home_or_visitor_win,last_home_and_visitor_score_diff,home_to_last_distance,visitor_to_last_distance,date_diff_between_two_team,home_last_matches_win,visitor_last_matches_win
0,0,7:00 PM,Orlando Magic,87.0,Indiana Pacers,97.0,Box Score,NaN,"18,165",True,...,0,8,29,False,-21,0,0,0,0,0
1,1,10:30 PM,Los Angeles Clippers,103.0,Los Angeles Lakers,116.0,Box Score,NaN,"18,997",True,...,0,12,4,True,8,0,0,0,0,0
2,2,8:00 PM,Chicago Bulls,95.0,Miami Heat,107.0,Box Score,NaN,"19,964",True,...,0,0,10,False,-10,0,0,0,0,0
3,3,7:00 PM,Brooklyn Nets,94.0,Cleveland Cavaliers,98.0,Box Score,NaN,"20,562",True,...,0,27,7,True,20,0,0,0,0,0
4,4,8:30 PM,Atlanta Hawks,109.0,Dallas Mavericks,118.0,Box Score,NaN,"19,834",True,...,0,16,13,True,3,0,0,0,0,0
5,5,7:30 PM,Washington Wizards,102.0,Detroit Pistons,113.0,Box Score,NaN,"19,258",True,...,0,22,23,False,-1,0,0,0,0,0
6,6,10:30 PM,Los Angeles Lakers,94.0,Golden State Warriors,125.0,Box Score,NaN,"19,596",True,...,True,9,12,False,-3,0,0,0,0,1
7,7,8:00 PM,Charlotte Bobcats,83.0,Houston Rockets,96.0,Box Score,NaN,"18,083",True,...,0,11,28,False,-17,0,0,0,0,0
8,8,8:00 PM,Orlando Magic,115.0,Minnesota Timberwolves,120.0,Box Score,OT,"17,988",True,...,False,21,29,False,-8,0,0,0,0,0
9,9,8:00 PM,Indiana Pacers,95.0,New Orleans Pelicans,90.0,Box Score,NaN,"17,803",False,...,True,25,8,True,17,0,0,0,0,1


In [6]:
from sklearn.preprocessing import LabelEncoder
#实例化一个LabelEncoding类
encoding = LabelEncoder()
#将字符串形式转化成整形数字形式
home_team_encoding = encoding.fit_transform(nba_data['Home Team'].values)#transformer need fit() and transform() or fit_transform() these methods
visitor_team_encoding = encoding.fit_transform(nba_data['Visitor Team'].values)#transformer need fit() and transform() or fit_transform() these methods
# temp = encoding.fit_transform(['a','b','c','d','e','f'])#还可以这样用
# print set(home_team_encoding)
# print set(visitor_team_encoding)
X_teams = np.stack([home_team_encoding,visitor_team_encoding],axis=0).T#vstack 即为stack(axis = 0),hstack 为stack(axis = 1)
print (X_teams)#转化成数值表示的一组team
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder()
X_teams_onehot = onehot.fit_transform(X_teams).toarray()

print (X_teams_onehot)#转换成onehot形式的一组team
print (X_teams_onehot.shape)

[[11 21]
 [13 12]
 [15  4]
 ...
 [15 26]
 [15 26]
 [26 15]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(1319, 60)


In [7]:
X_train = np.hstack([X_teams_onehot,nba_data.ix[:,['HT last win', 'VT last win',
       'last_home_Rk', 'last_visitor_Rk', 'last_home_or_visitor_win',
       'last_home_and_visitor_score_diff',
         'date_diff_between_two_team',
        'home_last_matches_win',
        'visitor_last_matches_win']]])#增加home_last_matches_win,visitor_last_matches_win这两个特征并没有使得预测准确率上升
# print (X_train.shape)
# y_train = np.hstack([nba_data.ix[:,'win_type'],1-nba_data.ix[:,'win_type']])
y_train = np.vstack([np.array(nba_data.ix[:,'win_type']),1-np.array(nba_data.ix[:,'win_type'])]).T#表示home team win的概率和visitor team win的概率
print (X_train.shape[1])
print (y_train)


69
[[1 0]
 [1 0]
 [1 0]
 ...
 [0 1]
 [0 1]
 [1 0]]


In [ ]:
X_train = np.array(X_train)
print (X_train.shape[1])
for index in range(X_train.shape[1]):
    X_train[:,index] /= max(X_train[:,index])#数据归一化


69


In [ ]:
'''用三层的神经网络拟合预测比随机森林略好,使用以下参数构建的神经网络预测正确率突破70%'''
import tflearn
net = tflearn.input_data(shape=[None,X_train.shape[1]])
net = tflearn.fully_connected(net,256,activation='sigmoid')
net = tflearn.fully_connected(net,32,activation='sigmoid')
net = tflearn.fully_connected(net,2,activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(X_train,y_train,n_epoch=50,batch_size=30,show_metric=True)


Training Step: 2060  | total loss: 0.56836 | time: 0.200s
| Adam | epoch: 047 | loss: 0.56836 - acc: 0.7043 -- iter: 1080/1319
